In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random

In [2]:
df_sp = pd.read_csv(r'F:\Abhilah_First\Results\Observed_T\noised_stake_data.csv')
df_sp.head(4)

,Unnamed: 0,Julian Day,obs_period(days),elevation(m),Derbis thickness (cm),melt (cm),pdd,Model-1 Melt Rate (cm/day),Model-2 Melt Rate (cm/day),Model-3 Melt Rate (cm/day)
0,0,232,17,3868.0,110.112269,9.226452,186.225022,0.975783,0.530753,0.657459
1,1,250,18,3867.0,113.032401,9.876500,155.421463,0.760865,0.408535,0.509296
2,2,263,13,3867.0,114.128684,1.758438,110.102815,0.743350,0.397225,0.496367
3,3,273,10,3868.0,116.278306,3.607321,79.189979,0.689702,0.365156,0.458390


In [3]:
df_model1_param = pd.read_csv(r'F:\Abhilah_First\Results\Observed_T\Parameters_1000.csv')
df_model1_param.head()

,Unnamed: 0,TF1 Model 1,TF2 Model 1,MSE Model 1,DDF Model 2,d0 Model 2,MSE Model 2,DDF Model 3,d0 Model 3,a Model 3,MSE Model 3
0,0,1.377083,-0.193208,0.628261,4.882812,12.876337,0.427875,4.344631,6.342217,0.352784,0.427687
1,1,1.029540,-0.380523,0.534095,4.590246,15.585462,0.478000,3.862951,10.103826,0.408360,0.473450
2,2,0.974426,-0.384808,0.469312,4.490768,14.961118,0.422020,3.831872,8.800334,0.382067,0.417324
3,3,1.065527,-0.355470,0.513937,4.848621,13.643871,0.424900,4.073580,8.720120,0.401118,0.421400
4,4,1.029322,-0.359518,0.516735,4.687397,13.981979,0.442138,4.059110,8.195743,0.390163,0.436305


In [4]:
data = {'TF1':df_model1_param['TF1 Model 1']} 
df_all_iterations = pd.DataFrame(data)
df_all_iterations['TF2'] = df_model1_param['TF2 Model 1']
MSEs = list(df_model1_param['MSE Model 1'])
df_all_iterations['MSE'] = df_model1_param['MSE Model 1']
weight = []
for i in MSEs:
    weight.append(math.exp(-i))
df_all_iterations['Weight'] = weight
df_all_iterations

,TF1,TF2,MSE,Weight
0,1.377083,-0.193208,0.628261,0.533519
1,1.029540,-0.380523,0.534095,0.586200
2,0.974426,-0.384808,0.469312,0.625433
3,1.065527,-0.355470,0.513937,0.598136
4,1.029322,-0.359518,0.516735,0.596465
...,...,...,...,...
995,1.336242,-0.223546,0.590772,0.553900
996,1.256383,-0.244990,0.572308,0.564221
997,1.227578,-0.267807,0.536815,0.584607
998,0.956127,-0.400747,0.438141,0.645235


In [5]:
print(min(weight), max(weight))
count = 0
for i in weight:
    if i <= 0.655:
        count +=1
print('count: ', count) 
# there is no zero weight, number of non zero weights is 1000

0.4872556274383204 0.6989493321830083
count:  973


In [6]:
df_all_iterations['TF1*Weight'] = df_all_iterations['TF1']*df_all_iterations['Weight']
df_all_iterations['TF2*Weight'] = df_all_iterations['TF2']*df_all_iterations['Weight']

df_all_iterations['Weight*(TF1 - xBARw)^2'] = df_all_iterations['Weight']*((df_all_iterations['TF1'] - sum(df_all_iterations['TF1*Weight']/1000) )**2)
df_all_iterations['Weight*(TF2 - yBARw)^2'] = df_all_iterations['Weight']*((df_all_iterations['TF2'] - sum(df_all_iterations['TF2*Weight']/1000) )**2)

df_all_iterations.head(4)

,TF1,TF2,MSE,Weight,TF1*Weight,TF2*Weight,Weight*(TF1 - xBARw)^2,Weight*(TF2 - yBARw)^2
0,1.377083,-0.193208,0.628261,0.533519,0.734700,-0.103080,0.275659,4.722907e-08
1,1.029540,-0.380523,0.534095,0.586200,0.603516,-0.223062,0.080799,2.063323e-02
2,0.974426,-0.384808,0.469312,0.625433,0.609438,-0.240671,0.062512,2.303118e-02
3,1.065527,-0.355470,0.513937,0.598136,0.637330,-0.212620,0.099202,1.580610e-02


In [7]:
TF1_variability = (sum(df_all_iterations['Weight*(TF1 - xBARw)^2'])*1000/(1000*(1000-1)*sum(df_all_iterations['Weight'])) )**(0.5)


TF2_variability = (sum(df_all_iterations['Weight*(TF2 - yBARw)^2'])*1000/(1000*(1000-1)*sum(df_all_iterations['Weight'])) )**(0.5)

print(df_all_iterations['TF1'][MSEs.index(min(MSEs))], df_all_iterations['TF2'][MSEs.index(min(MSEs))])
print(round(TF1_variability,3), round(TF2_variability,3))
print('RMSE =',min(MSEs)**0.5)

0.8237190657685329 -0.5002518958992125
0.015 0.005
RMSE = 0.5984789265487266
